In [9]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.measure import label, regionprops
from skimage.measure import moments_hu
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import defaultdict
import shutil
import hashlib

Définitions des fonctions statistiques et graphiques des caractéristiques

In [10]:
df = pd.read_csv("/workspaces/datasciencetest_reco_plante/notebooks/Plant_V_Seg_all_features.csv")
df.head()

,ID_Image,Est_Saine,is_black,dimensions,aire,périmètre,circularité,excentricité,aspect_ratio,mean_R,...,disease_Leaf_blight_(Isariopsis_Leaf_Spot),disease_Leaf_scorch,disease_Northern_Leaf_Blight,disease_Powdery_mildew,disease_Septoria_leaf_spot,disease_Spider_mites Two-spotted_spider_mite,disease_Target_Spot,disease_Tomato_Yellow_Leaf_Curl_Virus,disease_Tomato_mosaic_virus,disease_healthy
0,1,0,0,27x53,549.5,224.208151,0.137365,0.908846,0.509434,0.051777,...,False,False,False,False,False,False,False,False,True,False
1,2,0,0,115x94,2425.0,1143.082383,0.023322,0.860274,1.223404,0.084200,...,False,False,False,False,False,False,False,False,True,False
2,3,0,0,73x78,1662.0,807.092487,0.032062,0.841733,0.935897,0.057895,...,False,False,False,False,False,False,False,False,True,False
3,4,0,0,142x170,8371.5,2032.642906,0.025462,0.710637,0.835294,0.107897,...,False,False,False,False,False,False,False,False,True,False
4,5,0,0,31x31,247.5,155.296463,0.128962,0.934820,1.000000,0.070700,...,False,False,False,False,False,False,False,False,True,False


In [11]:
df.columns

Index(['ID_Image', 'Est_Saine', 'is_black', 'dimensions', 'aire', 'périmètre',
       'circularité', 'excentricité', 'aspect_ratio', 'mean_R', 'mean_G',
       'mean_B', 'std_R', 'std_G', 'std_B', 'contrast', 'energy',
       'homogeneity', 'dissimilarite', 'Correlation', 'contour_density',
       'mean_H', 'mean_S', 'mean_V', 'netteté', 'hu_1', 'hu_2', 'hu_3', 'hu_4',
       'hu_5', 'hu_6', 'hu_7', 'plant_Apple', 'plant_Blueberry',
       'plant_Cherry_(including_sour)', 'plant_Corn_(maize)', 'plant_Grape',
       'plant_Orange', 'plant_Peach', 'plant_Pepper,_bell', 'plant_Potato',
       'plant_Raspberry', 'plant_Soybean', 'plant_Squash', 'plant_Strawberry',
       'plant_Tomato', 'disease_Apple_scab', 'disease_Bacterial_spot',
       'disease_Black_rot', 'disease_Cedar_apple_rust',
       'disease_Cercospora_leaf_spot Gray_leaf_spot', 'disease_Common_rust_',
       'disease_Early_blight', 'disease_Esca_(Black_Measles)',
       'disease_Haunglongbing_(Citrus_greening)', 'disease_Late

In [12]:
def add_category_columns(df):
    """
    À appeler une seule fois pour dériver :
     - nom_plante   : à partir des colonnes plant_*
     - nom_maladie  : à partir des colonnes disease_*
     - est_saine_lbl: version labelisée de Est_Saine (0/1 → 'malade'/'saine')
    """
    df = df.copy()
    # Espèce
    plant_cols = [c for c in df.columns if c.startswith("plant_")]
    df["nom_plante"] = (
        df[plant_cols]
        .idxmax(axis=1)
        .str.replace("plant_", "")
        .str.replace("_", " ")
    )
    # Maladie
    disease_cols = [c for c in df.columns if c.startswith("disease_")]
    df["nom_maladie"] = (
        df[disease_cols]
        .idxmax(axis=1)
        .str.replace("disease_", "")
        .str.replace("_", " ")
    )
    # Label santé
    df["est_saine_lbl"] = df["Est_Saine"].map({0: "malade", 1: "saine"})
    return df


In [13]:
# Analyse statistique - Violinplot interactif Plotly (distribution par classe)
def plot_violin_interactive(df, feature, by="nom_plante"):
    """
    Trace un violon Plotly de la distribution de `feature` 
    groupée par la colonne `by` (nom_plante, nom_maladie ou est_saine_lbl).
    """
    # S’il manque encore les colonnes de catégorie, on les crée
    needed = {"nom_plante", "nom_maladie", "est_saine_lbl"}
    if not needed.issubset(df.columns):
        df = add_category_columns(df)
    
    if by not in df.columns:
        raise ValueError(f"La colonne de groupe '{by}' n'existe pas dans le df.")
    
    fig = px.violin(
        df,
        y=feature,
        x=by,
        box=True,
        points="all",
        hover_data=df.columns,
        title=f"Distribution de {feature} par {by}"
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

In [16]:
def plot_box_interactive(df, feature, by="nom_plante"):
    """
    Trace un box plot Plotly de la distribution de `feature` 
    groupée par la colonne `by` (nom_plante, nom_maladie ou est_saine_lbl).
    
    Params:
    - df      : pandas DataFrame, éventuellement enrichi par add_category_columns
    - feature : str, nom de la colonne numérique à tracer
    - by      : str, nom de la colonne de regroupement
                (par défaut "nom_plante", ou "nom_maladie", "est_saine_lbl")
    
    Usage:
    df2 = add_category_columns(df)
    plot_box_interactive(df2, feature="aire", by="est_saine_lbl")
    """
    # Vérifier et ajouter les colonnes de catégorie si besoin
    needed = {"nom_plante", "nom_maladie", "est_saine_lbl"}
    if not needed.issubset(df.columns):
        df = add_category_columns(df)
    
    if feature not in df.columns:
        raise ValueError(f"La feature '{feature}' n'existe pas dans le DataFrame.")
    if by not in df.columns:
        raise ValueError(f"La colonne de groupe '{by}' n'existe pas dans le DataFrame.")
    
    fig = px.box(
        df,
        x=by,
        y=feature,
        points="all",             # afficher tous les points
        hover_data=df.columns,    # inclure toutes les colonnes en tooltip
        title=f"Box plot de {feature} par {by}"
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

In [ ]:
# Représenter des features de forme selon l’espèce (nom_plante)
plot_violin_interactive(df, feature='aire', by='nom_plante')

In [ ]:
# Représenter des features de forme selon l’espèce (nom_plante)
plot_box_interactive(df, feature='aspect_ratio', by='nom_plante')

In [ ]:
# Représenter des features de forme selon l’espèce (nom_plante)
plot_violin_interactive(df, feature='excentricité', by='nom_plante')

In [ ]:
# Représenter des features de forme selon l’espèce (nom_plante)
plot_violin_interactive(df, feature='circularité', by='nom_plante')